# Lab 2.3: Human-in-the-Loop - Reviewing Tool Calls

In this lab, we will implement a critical safety pattern: review tool calls before they execute. This is common for sensitive actions (e.g., writing to a database, sending emails).

## Pattern
1. **Agent** decides to call a tool.
2. **Interrupt** before the `tools` node.
3. **Human** inspects the call.
4. **Action**:
    - **Approve**: Resume execution.
    - **Modify**: Update the tool arguments.
    - **Reject**: Cancel and give feedback to the agent.

In [ ]:
# 1. Install Dependencies
%pip install -qU langchain-groq langchain-community langgraph

In [1]:
# 2. Setup API Keys
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

## 3. Define Tools and Agent
We create a sensitive tool, e.g., `delete_user`.

In [2]:
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

@tool
def delete_user(user_id: str):
    """Deletes a user from the database. Use with caution."""
    print(f"!!! DELETING USER {user_id} !!!")
    return f"User {user_id} deleted successfully."

# Setup Agent
# Initialize LLM
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)
memory = MemorySaver()

# Create Graph with Interrupt
# We interrupt before the 'tools' node executes
graph = create_react_agent(
    llm, 
    tools=[delete_user], 
    checkpointer=memory,
    interrupt_before=["tools"]
)

d:\Projects\GenAI Labs\Jan-2026-day2-labs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Prathamesh Bonde\AppData\Local\Temp\ipykernel_16852\1588312709.py:23: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = create_react_agent(


## 4. Run Agent (Trigger Tool)
We ask the agent to delete a user.

In [3]:
thread_config = {"configurable": {"thread_id": "review-demo-1"}}

print("--- Sending Request ---")
result = graph.invoke(
    {"messages": [("user", "Please delete user 12345")]}, 
    config=thread_config
)

print("--- Execution Paused ---")

--- Sending Request ---
--- Execution Paused ---


## 5. Inspect Pending Action
The graph is paused. Let's see what it wants to do.

In [4]:
state = graph.get_state(thread_config)
last_message = state.values["messages"][-1]

# Check if there are tool calls
if last_message.tool_calls:
    print("Pending Tool Calls:")
    for tc in last_message.tool_calls:
        print(f"- Tool: {tc['name']}, Args: {tc['args']}")

Pending Tool Calls:
- Tool: delete_user, Args: {'user_id': '12345'}


## 6. Decision: Approve or Reject
Here we can choose to just resume (approve) or update state (reject).

In [5]:
# APPROVE: Just run invoke(None)
# graph.invoke(None, config=thread_config)

# REJECT: We can provide feedback by adding a message pretending to be the 'tool output' or just a user message.
# A cleaner way often involves using a function to reject, but for simplicity, let's just resume to show it works.

print("--- Approving Execution ---")
final_result = graph.invoke(None, config=thread_config)

print("Final Response:", final_result["messages"][-1].content)

--- Approving Execution ---
!!! DELETING USER 12345 !!!
Final Response: User 12345 has been deleted from the database. This action is irreversible, so please ensure this was intentional. Let me know if you need further assistance!
